In [2]:
import numpy as np
import pandas as pd
import os
import geoplot as gplt
import geopandas as geopd
import geoplot.crs as gcrs
from numpy.ma.extras import unique

/Users/jwilliamson/venvs/Apple-mapping/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
town_cords = pd.read_csv("/Users/jwilliamson/git/plasma_particle_simulator/Apple-mapping/codes/town cords.csv", index_col=False)
town_cords.head(10)

,Location,Latitude,Longitude
0,Anson,44.80,-69.90
1,Ashland,46.62,-68.40
2,Auburn city,44.09,-70.25
3,Augusta city,44.33,-69.73
4,Bangor city,44.83,-68.79
5,Bar Harbor,44.38,-68.21
6,Bath city,43.93,-69.83
7,Belfast city,44.43,-69.03
8,Berwick,43.27,-70.86
9,Biddeford city,43.44,-70.40


In [4]:
county_cords = pd.read_csv("/Users/jwilliamson/git/plasma_particle_simulator/Apple-mapping/cord data/CountyCords.csv", index_col=False)
county_cords

,County,Latitude,longitude
0,Androscoggin,-70.198,44.187
1,Aroostook,-68.412,46.574
2,Cumberland,-70.268,43.874
3,Franklin,-70.449,44.980
4,Hancock,-68.402,44.614
5,Kennebec,-69.753,44.415
6,Knox,-69.135,44.101
7,Lincoln,-69.518,44.078
8,Oxford,-70.734,44.426
9,Penobscot,-68.575,45.354


In [5]:
def clean_country(value):
    if isinstance(value, str):
        return value.lower().replace("country", "").strip()

In [6]:
def clean_city(value):
    if isinstance(value, str):
        return value.lower().replace("city", "").strip()
    return value

In [7]:
town_cords_clean = town_cords
town_cords_clean['Location'] = town_cords["Location"].apply(clean_city)
town_cords_clean

,Location,Latitude,Longitude
0,anson,44.80,-69.90
1,ashland,46.62,-68.40
2,auburn,44.09,-70.25
3,augusta,44.33,-69.73
4,bangor,44.83,-68.79
...,...,...,...
125,winthrop,44.31,-69.96
126,wiscasset,44.01,-69.67
127,woodland,45.16,-67.41
128,yarmouth,43.80,-70.19


In [8]:
source_df = pd.read_csv("/Users/jwilliamson/git/plasma_particle_simulator/Apple-mapping/csv/reformated_sources.csv", index_col=False, keep_default_na=False)
source_df

EmptyDataError: No columns to parse from file

In [130]:
def clean_location(value):
    if isinstance(value, str):
        return (value.lower()
                 .replace("foxcroft", "dover-foxcroft")
                 .strip())
    return value

In [131]:
clean_source_df = source_df
clean_source_df['Location'] = clean_source_df["Location"].apply(clean_location)
mask = clean_source_df["Location"] == ""
clean_source_df["Missing Location"] = mask
clean_source_df


In [132]:
nonull_loc_source_df = clean_source_df[~mask]
nonull_loc_source_df

In [133]:
merge_nonnull = nonull_loc_source_df.merge(town_cords, on=["Location"], how="left")
merge_nonnull

In [149]:
null_loc_source_df = clean_source_df[mask]
null_loc_source_df

In [164]:

nonull_loc_source_df[["Location"]].drop_duplicates().merge(town_cords_clean ,on="Location", how="left" ).to_csv("that.csv", index=False)

In [156]:
town_cords_clean